# <p style='text-align: center;'> Проект по SQL</p><a class="tocSkip">

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Цели исследования

Проанализировать базу данных. Эти данные помогут сформулировать ценностное предложение для нового продукта.
* объем выпуска за последние 20 лет
* для каждой книги посчитать количество обзоров и среднюю оценку
* определить самое выпускающее издательство
* определить самого популярного автора
* оценить количество обзоров от пользователей, которые ставят наибольшее количество оценок

In [8]:
books = pd.io.sql.read_sql('''
                          SELECT * 
                          FROM books
                       ''', 
                       con = engine)
display(books.head())
books.info()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [10]:
authors = pd.io.sql.read_sql('''
                          SELECT * 
                          FROM authors
                       ''', 
                       con = engine)
display(authors.head())
authors.info()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.0+ KB


In [11]:
reviews = pd.io.sql.read_sql('''
                          SELECT * 
                          FROM reviews
                       ''', 
                       con = engine)
display(reviews.head())
reviews.info()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [12]:
ratings = pd.io.sql.read_sql('''
                                SELECT * 
                                FROM ratings
                             ''', 
                             con = engine)
display(ratings.head())
ratings.info()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.8+ KB


In [13]:
publishers = pd.io.sql.read_sql('''
                                   SELECT * 
                                   FROM publishers
                                ''', 
                                con = engine)
display(publishers.head())
publishers.info()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


## Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
## сколько книг вышло после 1 января 2000 года
pd.io.sql.read_sql('''
                      SELECT COUNT(book_id) 
                      FROM books
                      WHERE publication_date > '2000-01-01'
                   ''', 
                   con = engine)

,count
0,819


## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [22]:
df = pd.io.sql.read_sql('''
                      SELECT b.book_id,
                             b.title,
                             COUNT(DISTINCT(rws.review_id)) AS rewiews_count,
                             ROUND(AVG(rtng.rating), 2) AS AVG_rating
                      FROM books AS b
                      LEFT OUTER JOIN reviews AS rws ON rws.book_id = b.book_id
                      LEFT OUTER JOIN ratings AS rtng ON rtng.book_id = b.book_id
                      GROUP BY b.book_id
                      ORDER BY rewiews_count DESC
                   ''', 
                   con = engine)
print(df.rewiews_count.sum())
display(df.head())

2793


,book_id,title,rewiews_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08


In [23]:
df = pd.io.sql.read_sql('''
                      SELECT b.book_id,
                             b.title,
                             COUNT(DISTINCT(rws.review_id)) AS rewiews_count,
                             ROUND(AVG(rtng.rating), 2) AS AVG_rating
                      FROM books AS b
                      LEFT OUTER JOIN reviews AS rws ON rws.book_id = b.book_id
                      LEFT OUTER JOIN ratings AS rtng ON rtng.book_id = b.book_id
                      GROUP BY b.book_id
                      ORDER BY avg_rating DESC
                   ''', 
                   con = engine)
print(df.rewiews_count.sum())
display(df.head())

2793


,book_id,title,rewiews_count,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.0
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.0
2,390,Light in August,2,5.0
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.0
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.0


**Вывод:**

Одни из лучших книг по количеству обзоров: 

* Twilight
* Water for Elephants
* The Glass Castle
* Harry Potter and the Prisoner of Azkaba
* the curious Incident of the Dog in the Night-Time

Одни из лучших книг по рейтингу: 

* Arrows of the Queen (Heralds of Valdemar #1)
* The Walking Dead Book One (The Walking Dead
* Light in August	
* Wherever You Go There You Are: Mindfulness Me...
* Captivating: Unveiling the Mystery of a Woman'...

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [9]:
pd.io.sql.read_sql('''
                      SELECT pb.publisher,
                             COUNT(b.book_id) AS books_count
                      FROM books AS b
                      LEFT OUTER JOIN publishers AS pb ON pb.publisher_id = b.publisher_id
                      WHERE b.num_pages > 50
                      GROUP BY pb.publisher
                      ORDER BY books_count DESC
                      LIMIT 1
                   ''', 
                   con = engine)

,publisher,books_count
0,Penguin Books,42


## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [26]:
pd.io.sql.read_sql('''
                      SELECT a.author
                      
                      FROM books AS b
                      LEFT OUTER JOIN authors AS a ON a.author_id = b.author_id
                      LEFT OUTER JOIN ratings AS rtng ON rtng.book_id = b.book_id
                      WHERE b.book_id IN (SELECT b.book_id
                                         FROM books AS b
                                         LEFT OUTER JOIN ratings AS rtng ON rtng.book_id = b.book_id
                                         GROUP BY b.book_id
                                         HAVING COUNT(rtng.rating) >= 50)
                      GROUP BY a.author
                      ORDER BY AVG(rtng.rating) DESC
                      LIMIT 1
                   ''', 
                   con = engine)

,author
0,J.K. Rowling/Mary GrandPré


## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [28]:
pd.io.sql.read_sql('''SELECT ROUND(AVG(rw_count)) AS AVG_rw_count
                      FROM(SELECT COUNT(DISTINCT(rws.review_id)) AS rw_count
                             
                           FROM reviews AS rws
                           LEFT OUTER JOIN books AS b ON rws.book_id = b.book_id
                           LEFT OUTER JOIN ratings AS rtng ON rtng.book_id = b.book_id
                           WHERE rws.username IN (SELECT rtng.username
                                                  FROM ratings AS rtng
                                                  GROUP BY rtng.username
                                                  HAVING COUNT(rtng.rating) > 50)
                           GROUP BY rws.username) AS q_1
                   ''', 
                   con = engine)

,avg_rw_count
0,24.0


## Выводы:

1. после 1 января 2000 года вышло 819 книг
2. для каждой из книг определили количество отзывов и среднюю оценку
3. наибольшее число книг выпустила Penguin Books
4. J.K. Rowling/Mary GrandPré - автор книг с самой высой оценкой (из тех, что получили более 50 оценок)
5. пользователи, которые поставили больше 50 оценок, в среднем написали по 215 отзыввов